# ChemBERT_MTR

## 라이브러리(restart)

In [ ]:
# 런타임 재시작
!pip install -U "numpy==1.26.4"

## 라이브러리2

In [ ]:
# 드라이브 마운트 (colab에서만)
'''from google.colab import drive
drive.mount('/content/drive')'''

In [ ]:
# (A) 우리 작업에 불필요하고 버전 충돌의 원인이 되는 패키지 제거
!pip uninstall -y bigframes peft diffusers gradio gcsfs cuml-cu12 umap-learn || true

# (B) ChemBERTa에 필요한 핵심 스택만 확정 설치 (서로 호환되는 조합)
!pip install --no-cache-dir \
  "transformers==4.44.2" \
  "tokenizers==0.19.1" \
  "accelerate==0.34.2" \
  "huggingface_hub==0.24.6" \
  "datasets==2.20.0" \
  "scikit-learn==1.6.1" \
  "torchmetrics==1.4.0" \
  "sentencepiece==0.1.99"

!pip install -U rdkit

In [ ]:
# 라이브러리
import os, json, random, math, time
import numpy as np
import pandas as pd
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import torch
from torch import nn
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, set_seed
)
from rdkit import Chem
from transformers import EarlyStoppingCallback

In [ ]:
# ===== 기본 설정 =====
SEED = 42
set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
N_FOLDS = 5
OUTPUT_ROOT = "./chemberta_5fold"

# ===== 스케줄러 & 얼리 스톱 설정 =====
LR_SCHEDULER = "cosine_with_restarts"
ES_PATIENCE = 5
ES_DELTA = 1e-4

## 데이터

In [ ]:
# ===== 유틸 =====
def IC50_to_pIC50(ic50_nM):
    ic50_nM = np.clip(ic50_nM, 1e-10, None)
    return 9 - np.log10(ic50_nM)

def pIC50_to_IC50(pIC50):
    return np.power(10.0, 9 - pIC50)

def canonicalize_smiles(smi: str):
    if pd.isna(smi):
        return None
    mol = Chem.MolFromSmiles(str(smi))
    return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=True) if mol else None

In [ ]:
# ===== 데이터 전처리 =====
chembl_ask = pd.read_csv('./ChEMBL_ASK1(IC50).csv', sep=";")
pubchem_ask = pd.read_csv('./Pubchem_ASK1.csv')

chembl_ask.dropna(subset=['Standard Value'],inplace=True)
chembl_ask = chembl_ask[chembl_ask['Standard Relation'] == "'='"]
chembl_ask.reset_index(drop=True,inplace=True)

pubchem_ask.dropna(subset=['Activity_Value'],inplace=True)
pubchem_ask = pubchem_ask[pubchem_ask['Activity_Type']=='IC50']
pubchem_ask['Activity_Qualifier'] = pubchem_ask['Activity_Qualifier'].fillna('=')
pubchem_ask = pubchem_ask[pubchem_ask['Activity_Qualifier'] == '=']
pubchem_ask.reset_index(drop=True,inplace=True)

df_names = pd.read_excel('./CAS_KPBMA_MAP3K5_IC50s.xlsx', sheet_name='Ligand Number Names SMILES', header=1)
df_ic50  = pd.read_excel('./CAS_KPBMA_MAP3K5_IC50s.xlsx', sheet_name='MAP3K5 Ligand IC50s', header=1)
cas_df = pd.merge(df_names, df_ic50, on="Substance Name", how="inner")
cas_df['Measurement Prefix (Parsed)'] = cas_df['Measurement Prefix (Parsed)'].fillna('=')
cas_df = cas_df[cas_df['Measurement Prefix (Parsed)'] == '=']
cas_df = cas_df[cas_df['Assay Parameter'] == 'IC50']
cas_df.reset_index(drop=True,inplace=True)

In [ ]:
# ===== 데이터 처리 =====
chem_df = chembl_ask[['Smiles','Standard Value']].copy()
chem_df.rename(columns={'Standard Value':'ic50_nM'}, inplace=True)
chem_df.dropna(inplace=True)
chem_df['ic50_nM'] = pd.to_numeric(chem_df['ic50_nM'], errors='coerce')

pub_df = pubchem_ask[['SMILES','Activity_Value']].copy()
pub_df.rename(columns={'SMILES':'Smiles', 'Activity_Value':'ic50_nM'}, inplace=True)
pub_df.dropna(inplace=True)
pub_df['ic50_nM'] = pd.to_numeric(pub_df['ic50_nM'], errors='coerce')
# 가정: PubChem 값이 μM → nM 로 변환
pub_df['ic50_nM'] = pub_df['ic50_nM'] * 1000

# cas_df
cas_df = cas_df[['SMILES_x', 'Single Value (Parsed)']].rename(columns={'SMILES_x':'Smiles','Single Value (Parsed)':'ic50_nM'})
cas_df.dropna(inplace=True)
cas_df['ic50_nM'] = pd.to_numeric(cas_df['ic50_nM'], errors='coerce')
# 가정: μM → nM
cas_df['ic50_nM'] = cas_df['ic50_nM'] * 1000
cas_df.reset_index(drop=True, inplace=True)

# 병합
train_df = pd.concat([chem_df, pub_df, cas_df], axis=0, ignore_index=True)
train_df = train_df.dropna(subset=['Smiles','ic50_nM']).copy()
train_df['Smiles'] = train_df['Smiles'].map(canonicalize_smiles)
train_df = train_df.dropna(subset=['Smiles']).reset_index(drop=True)

print("Train size:", len(train_df))

In [ ]:
# pIC50 변환 → 구간 필터 → 동일 SMILES 중앙값 집계
train_df['pIC50'] = train_df['ic50_nM'].apply(IC50_to_pIC50)
train_df = train_df[(train_df['pIC50'] >= 6) & (train_df['pIC50'] <= 10)]
train_df = (train_df.groupby('Smiles', as_index=False)['pIC50']
                     .median()
                     .reset_index(drop=True))

print("Train size:", len(train_df))

## 모델링

In [ ]:
# ======= 여기서부터 '1) Scaffold 기반 StratifiedGroupKFold' 적용 =======

# 1) scaffold 생성 함수
def smiles_to_scaffold(smi: str):
    m = Chem.MolFromSmiles(smi)
    return MurckoScaffoldSmiles(mol=m, includeChirality=True) if m else "NONE"

# 2) 층화용 pIC50 bin + 그룹용 scaffold
#   - q=10으로 기본 설정, 값이 부족하면 자동으로 bin 수가 줄어듭니다(duplicates='drop').
train_df["bin"] = pd.qcut(train_df["pIC50"], q=10, labels=False, duplicates="drop")
train_df["scaffold"] = train_df["Smiles"].map(smiles_to_scaffold)

# 3) StratifiedGroupKFold로 splits 생성
cv = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
splits = list(cv.split(train_df, y=train_df["bin"], groups=train_df["scaffold"]))

In [ ]:
# ===== 모델/토크나이저 =====
model_name = "DeepChem/ChemBERTa-77M-MTR"
max_len = 256

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, num_labels=1, problem_type="regression")
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

def tok_with_label(ex):
    out = tokenizer(ex["Smiles"], max_length=max_len, truncation=True, padding="max_length")
    out["labels"] = float(ex["pIC50"])
    return out

In [ ]:
# ===== 메트릭(학습용) =====
def pIC50_to_IC50(pIC50):
    return np.power(10.0, 9 - pIC50)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds  = preds.reshape(-1).astype(np.float64)
    labels = labels.astype(np.float64)

    y_true = pIC50_to_IC50(labels); y_pred = pIC50_to_IC50(preds)
    rmse_ic50 = float(np.sqrt(np.mean((y_true - y_pred)**2)))
    rng = float(np.max(y_true) - np.min(y_true)); A = float(rmse_ic50 / (rng if rng>1e-12 else 1.0))

    x = labels - labels.mean(); y = preds - preds.mean()
    denom = float(np.sqrt((x**2).sum()) * np.sqrt((y**2).sum()))
    r = float((x*y).sum() / denom) if denom>0 else 0.0
    B = float(r*r)

    rmse = float(np.sqrt(np.mean((preds - labels)**2)))
    mae  = float(np.mean(np.abs(preds - labels)))
    ss_res = float(np.sum((labels - preds)**2)); ss_tot = float(np.sum((labels - labels.mean())**2))
    r2 = float(1.0 - ss_res/ss_tot) if ss_tot>0 else 0.0

    score = 0.4*(1.0 - min(A,1.0)) + 0.6*B
    return {"score": score, "A_nrmse": A, "B_r2": B, "rmse_ic50": rmse_ic50, "rmse": rmse, "mae": mae, "r2": r2}

In [ ]:
# ===== 학습 루프 =====
oof_pred_pic50 = np.zeros(len(train_df), dtype=np.float64)
fold_metrics = []

best_epochs, best_ckpts, best_scores = [], [], []

for fold, (trn_idx, val_idx) in enumerate(splits, 1):
    print(f"\n===== FOLD {fold}/{N_FOLDS} =====")
    df_trn = train_df.iloc[trn_idx].reset_index(drop=True)
    df_val = train_df.iloc[val_idx].reset_index(drop=True)

    ds_trn = Dataset.from_pandas(df_trn[['Smiles','pIC50']].copy()).map(tok_with_label)
    ds_val = Dataset.from_pandas(df_val[['Smiles','pIC50']].copy()).map(tok_with_label)
    for ds in (ds_trn, ds_val):
        ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])

    model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

    out_dir = os.path.join(OUTPUT_ROOT, f"fold{fold}")
    args = TrainingArguments(
        output_dir=out_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        num_train_epochs=50,
        save_total_limit=2,
        weight_decay=0.01,
        warmup_ratio=0.1,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="score",
        greater_is_better=True,
        fp16=True,
        logging_steps=50,
        lr_scheduler_type=LR_SCHEDULER,
        report_to="none",
        overwrite_output_dir=True,
        seed=SEED,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_trn,
        eval_dataset=ds_val,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # 폴드별 best epoch / ckpt / score 추출
    best_metric = -float("inf")
    best_epoch = None
    for log in trainer.state.log_history:
        if "eval_score" in log and log["eval_score"] > best_metric:
            best_metric = float(log["eval_score"])
            best_epoch = log.get("epoch", None)

    if best_epoch is None:
        last_eval = [l for l in trainer.state.log_history if "eval_score" in l]
        if len(last_eval) > 0:
            best_epoch = last_eval[-1].get("epoch", None)

    best_ckpt_path = trainer.state.best_model_checkpoint
    best_epochs.append(best_epoch)
    best_ckpts.append(best_ckpt_path)
    best_scores.append(best_metric)

    print(f"[FOLD {fold}] ✅ Best eval_score={best_metric:.6f} @ epoch={best_epoch} | ckpt={best_ckpt_path}")

    # OOF 채우기
    val_pred = trainer.predict(ds_val).predictions.reshape(-1)
    oof_pred_pic50[val_idx] = val_pred

    # 폴드 메트릭 기록
    m = trainer.evaluate(ds_val); fold_metrics.append(m)
    print({k: round(float(v), 5) for k,v in m.items()})


===== FOLD 1/5 =====


Map:   0%|          | 0/1131 [00:00<?, ? examples/s]

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Score,A Nrmse,B R2,Rmse Ic50,Rmse,Mae,R2
1,No log,57.882729,0.047563,959993.689173,0.079271,949913401.225838,7.608070,7.570441,-99.166729
2,59.425700,57.074448,0.094406,851029.191454,0.157343,842093070.937124,7.554763,7.517348,-97.767980
3,58.077100,55.261631,0.107970,651900.187780,0.179950,645054995.275129,7.433816,7.396833,-94.630878
4,58.077100,50.746826,0.100983,344456.372934,0.168304,340839453.923286,7.123681,7.087166,-86.817974
5,55.132700,36.432831,0.063105,90872.533958,0.105174,89918338.821657,6.035961,5.991433,-62.047421
6,40.624800,9.519953,0.031191,25693.008589,0.051986,25423222.519105,3.085442,2.923986,-15.474387
7,11.698100,3.088046,0.018076,4294.993235,0.030126,4249894.221061,1.757284,1.446710,-4.343900
8,11.698100,1.075627,0.008476,70.960939,0.014127,70215.822991,1.037124,0.811857,-0.861385
9,2.460600,0.660934,0.006511,1.528627,0.010852,1512.575716,0.812979,0.673861,-0.143755
10,0.722100,0.707101,0.039024,0.923149,0.013806,913.455761,0.840893,0.680291,-0.223646


...

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Score,A Nrmse,B R2,Rmse Ic50,Rmse,Mae,R2
1,No log,59.654087,0.062344,949644.029672,0.103907,949169207.835716,7.723606,7.679894,-86.565475
2,58.520600,58.920605,0.146298,853401.794193,0.243830,852975093.456031,7.675976,7.632701,-85.488787
3,58.520600,57.391861,0.182381,685701.493412,0.303968,685358642.794649,7.575742,7.533383,-83.244782
4,57.850700,53.965080,0.189007,429956.725252,0.315011,429741746.970060,7.346093,7.305599,-78.214646
5,53.725900,45.246403,0.188891,170186.604146,0.314818,170101510.875459,6.726545,6.689404,-65.416614
6,53.725900,22.864944,0.189875,46772.558769,0.316458,46749172.498505,4.781730,4.716554,-32.563154
7,38.120900,5.201656,0.197245,3472.776377,0.328742,3471039.989940,2.280714,2.048954,-6.635443
8,9.748800,1.226453,0.171196,37.240948,0.285327,37222.327556,1.107453,0.941816,-0.800294
9,9.748800,0.492723,0.458662,0.273008,0.279776,272.871426,0.701942,0.590646,0.276738
10,1.330300,0.484963,0.492251,0.211127,0.294503,211.021459,0.696393,0.552414,0.288130


[FOLD 5] ✅ Best eval_score=0.533364 @ epoch=12.0 | ckpt=/content/drive/MyDrive/jump_ai/chemberta_5fold/fold5/checkpoint-396


{'eval_loss': 0.43909, 'eval_score': 0.53336, 'eval_A_nrmse': 0.21619, 'eval_B_r2': 0.3664, 'eval_rmse_ic50': 216.07896, 'eval_rmse': 0.66264, 'eval_mae': 0.54429, 'eval_r2': 0.35546, 'eval_runtime': 0.2033, 'eval_samples_per_second': 1421.6, 'eval_steps_per_second': 24.595, 'epoch': 50.0}


In [ ]:
# ===== 결과 저장(OoF, 메트릭, 매니페스트) =====
y_true_pic50 = train_df["pIC50"].values.astype(np.float64)
oof_df = pd.DataFrame({"Smiles": train_df["Smiles"], "pIC50_true": y_true_pic50, "pIC50_oof": oof_pred_pic50})
oof_path = os.path.join(OUTPUT_ROOT, "oof_5fold.csv")
oof_df.to_csv(oof_path, index=False)
print("Saved OOF:", oof_path)

metrics_df = pd.DataFrame(fold_metrics)
metrics_csv = os.path.join(OUTPUT_ROOT, "fold_metrics.csv")
metrics_df.to_csv(metrics_csv, index=False)
print("Saved fold metrics:", metrics_csv)

In [ ]:
# manifest.json: 추론에서 사용할 정보(체크포인트/epoch/score 등)
manifest = {
    "created_at": int(time.time()),
    "model_name": model_name,
    "max_len": max_len,
    "n_folds": N_FOLDS,
    "folds": [
        {"fold": i+1, "checkpoint": ck, "best_epoch": (int(ep) if ep is not None and abs(ep-int(ep))<1e-9 else ep), "best_score": float(sc)}
        for i, (ep, ck, sc) in enumerate(zip(best_epochs, best_ckpts, best_scores))
    ],
    # 추론 스크립트에서 참고할 기본 경로(원하면 수정)
    "test_csv_path": "./test.csv",
    "submission_name": "submission_5fold.csv",
}
manifest_path = os.path.join(OUTPUT_ROOT, "manifest.json")
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)
print("Saved manifest:", manifest_path)

print("\n=== Per-fold best checkpoints (for inference) ===")
for f in manifest["folds"]:
    print(f"Fold {f['fold']}: epoch={f['best_epoch']}, score={f['best_score']:.6f}")
    print(f"           ckpt={f['checkpoint']}")